In [ ]:
from agentex import Agentex

client = Agentex(base_url="http://localhost:5003")


In [ ]:
AGENT_NAME = "at030-oai-deep-research"


In [ ]:
# (REQUIRED) Create a new task. For Agentic agents, you must create a task for messages to be associated with.
import uuid

rpc_response = client.agents.create_task(
    agent_name=AGENT_NAME,
    params={
        "name": f"{str(uuid.uuid4())[:8]}-task",
        "params": {}
    }
)

task = rpc_response.result
print(task)


In [ ]:
# Send an event to the agent

# The response is expected to be a list of TaskMessage objects, which is a union of the following types:
# - TextContent: A message with just text content   
# - DataContent: A message with JSON-serializable data content
# - ToolRequestContent: A message with a tool request, which contains a JSON-serializable request to call a tool
# - ToolResponseContent: A message with a tool response, which contains response object from a tool call in its content

# When processing the message/send response, if you are expecting more than TextContent, such as DataContent, ToolRequestContent, or ToolResponseContent, you can process them as well

rpc_response = client.agents.send_event(
    agent_name=AGENT_NAME,
    params={
        "content": {"type": "text", "author": "user", "content": "I want to research the latest developments in artificial intelligence and machine learning"},
        "task_id": task.id,
    }
)

event = rpc_response.result
print(event)


In [ ]:
# Subscribe to the async task messages produced by the agent
from agentex.lib.utils.dev_tools import subscribe_to_async_task_messages

task_messages = subscribe_to_async_task_messages(
    client=client,
    task=task, 
    only_after_timestamp=event.created_at, 
    print_messages=True,
    rich_print=True,
    timeout=10,  # Short timeout for clarifying questions
)


In [ ]:
# Respond to clarifying questions from the agent
# Replace this content with your actual response to the clarifying questions

clarification_response = """I'm specifically interested in:
1. Recent breakthroughs in large language models like GPT-4, Claude, and Gemini
2. Advances in computer vision and image generation models 
3. New developments in AI safety and alignment research
4. Applications of AI in scientific research and drug discovery
5. The latest funding rounds and company acquisitions in the AI space

I want a comprehensive report that covers both technical advances and business/industry developments from the past 6 months."""

rpc_response = client.agents.send_event(
    agent_name=AGENT_NAME,
    params={
        "content": {"type": "text", "author": "user", "content": clarification_response},
        "task_id": task.id,
    }
)

event = rpc_response.result
print(event)


In [ ]:
# Subscribe to messages after clarification response
from agentex.lib.utils.dev_tools import subscribe_to_async_task_messages

task_messages = subscribe_to_async_task_messages(
    client=client,
    task=task, 
    only_after_timestamp=event.created_at, 
    print_messages=True,
    rich_print=True,
    timeout=60,  # Longer timeout for deep research
)


In [ ]:
# Ask a follow-up question after the research is complete
follow_up_question = "Can you provide more details about the specific AI safety techniques that have been developed recently?"

rpc_response = client.agents.send_event(
    agent_name=AGENT_NAME,
    params={
        "content": {"type": "text", "author": "user", "content": follow_up_question},
        "task_id": task.id,
    }
)

event = rpc_response.result
print(event)


In [ ]:
# Subscribe to follow-up response
task_messages = subscribe_to_async_task_messages(
    client=client,
    task=task, 
    only_after_timestamp=event.created_at, 
    print_messages=True,
    rich_print=True,
    timeout=60,
)


In [ ]:
# Send responses to clarifying questions
# The agent will ask 2-3 questions - answer them one by one

# First clarification response
rpc_response = client.agents.send_event(
    agent_name=AGENT_NAME,
    params={
        "content": {"type": "text", "author": "user", "content": "I'm interested in recent hardware developments and companies working on quantum processors."},
        "task_id": task.id,
    }
)

event = rpc_response.result
print(event)


In [ ]:
# Subscribe to see the next clarifying question or research starting
task_messages = subscribe_to_async_task_messages(
    client=client,
    task=task, 
    only_after_timestamp=event.created_at, 
    print_messages=True,
    rich_print=True,
    timeout=10,
)


In [ ]:
# Second clarification response (if asked)
rpc_response = client.agents.send_event(
    agent_name=AGENT_NAME,
    params={
        "content": {"type": "text", "author": "user", "content": "Yes, I want to know about breakthroughs from 2024-2025, especially around error correction and qubit stability."},
        "task_id": task.id,
    }
)

event = rpc_response.result
print(event)


In [ ]:
# Subscribe to see the deep research results
# This will take longer as the agent performs multiple searches
task_messages = subscribe_to_async_task_messages(
    client=client,
    task=task, 
    only_after_timestamp=event.created_at, 
    print_messages=True,
    rich_print=True,
    timeout=60,  # Longer timeout for deep research
)


In [ ]:
# Optional: Ask a follow-up question after seeing the research
rpc_response = client.agents.send_event(
    agent_name=AGENT_NAME,
    params={
        "content": {"type": "text", "author": "user", "content": "Can you tell me more about IBM's recent quantum developments?"},
        "task_id": task.id,
    }
)

event = rpc_response.result

# Subscribe to see the follow-up research
task_messages = subscribe_to_async_task_messages(
    client=client,
    task=task, 
    only_after_timestamp=event.created_at, 
    print_messages=True,
    rich_print=True,
    timeout=60,
)
